目的：ポンプ選定のための、配管損失計算を行うプログラム
開発工程案
1. 損失計算機能の作成；Python
2. インターフェイスの作成（条件入力フォームなど）;JavaScript
3. 計算結果出力機能の作成（計算結果表、模式図の自動生成）;Python
4. 報告書の自動生成　；JavaScript？

実揚程

In [ ]:
#実揚程
hs = 7

配管摩擦損失　ヘーゼンウイリアムズ式

In [ ]:
#配管摩擦損失
hp = 3

曲がり、レジューサによる損失

In [ ]:
#曲がり、レジューサによる損失
he = 5

各種損失水頭の合計演算

In [ ]:
#合計損失
h = hs + hp + he

#損失計算の余裕率
eth = 0.1

hc = h*(1 + eth)

print(hc)

16.5
